In [7]:
import pandas as pd
import numpy as np
from pymongo import MongoClient
import requests
from bs4 import BeautifulSoup as bs
import re

import json
import pickle

base_url = "https://www.tripadvisor.com.tr/Restaurants-g293969-Turkey.html?page="


def getAndParse(url):
    response=requests.get(url, headers={"User-Agent": "Mozilla/5.0 (X11; CrOS x86_64 12871.102.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.141 Safari/537.36"})
    soup=bs(response.content, 'html.parser')
    return soup


# page_links = [] # first page
# page_links.append("https://www.tripadvisor.com/Restaurants-g186338-London_England.html#EATERY_LIST_CONTENTS")
page_links = [] # first page
page_links.append("https://www.tripadvisor.com/Restaurants-g494941-Indore_Indore_District_Madhya_Pradesh.html#EATERY_LIST_CONTENTS")
for i in range(30,19620,30):
    # page_links.append("https://www.tripadvisor.com.tr//Restaurants-g186338-oa"+str(i)+"-London_England.html#EATERY_LIST_CONTENTS")
    page_links.append("https://www.tripadvisor.com.tr//Restaurants-g494941-oa"+str(i)+"-Indore_Indore_District_Madhya_Pradesh.html#EATERY_LIST_CONTENTS")

cafe_names = []
cafe_links = []
cafe_tips = []



for page in range(5): # bütün restaurant linkleri burdan
    soup = getAndParse(page_links[page])
    cafe_names+=[x.text for x in soup.findAll("div",class_="OhCyu")]
    cafe_links+=['https://www.tripadvisor.com.tr'+x.a.get('href') for x in soup.findAll("div",class_="OhCyu")]    

# for i in cafe_names:
#     print(i)

df = pd.DataFrame()
df["cafe_names"] = cafe_names
df["cafe_links"] = cafe_links

# df.info()

df = df.drop_duplicates()
# df.info()

df = df.reset_index()
df = df.drop(columns=["index"])
# df.head()

review_links = [] # ***  -----
for i in range(len(df["cafe_links"])-140):
    soup = getAndParse(df["cafe_links"][i])
    review_link = ["https://www.tripadvisor.com/"+x.get("href") for x in soup.findAll("a",class_=re.compile("pageNum"))]
    review_links.append(dict([
        ('name', df["cafe_names"][i]),
        ('links', review_link)
        ]))
# with open('review_link_night2.pickle', 'wb') as handle:
#     pickle.dump(review_links, handle)


In [8]:
new_df = pd.DataFrame(data=review_links)
new_df = new_df[["name","links"]]
new_df

,name,links
0,1. Kebabsville,[https://www.tripadvisor.com//Restaurant_Revie...
1,2. Joshi Dahi Bada House,[https://www.tripadvisor.com//Restaurant_Revie...
2,3. Shreemaya Celebrity,[https://www.tripadvisor.com//Restaurant_Revie...
3,4. The Square - Sayaji Hotel,[https://www.tripadvisor.com//Restaurant_Revie...
4,5. 56 Dukan,[https://www.tripadvisor.com//Restaurant_Revie...
5,6. johny hot dog,[https://www.tripadvisor.com//Restaurant_Revie...
6,7. Nafees Restaurant,[https://www.tripadvisor.com//Restaurant_Revie...
7,8. Mediterra at Sayaji Hotel,[https://www.tripadvisor.com//Restaurant_Revie...
8,9. Restaurant Shreemaya,[https://www.tripadvisor.com//Restaurant_Revie...
9,10. Sanchi,[https://www.tripadvisor.com//Restaurant_Revie...


In [9]:

new = (new_df.groupby('name',as_index=False)
          .agg({'links':(lambda x: x)})
          .rename(columns={'links':'all_links'}))
l = new.all_links.to_list()
import numpy as np
r = []
for i in l:
    if(i==[]):
        r.append(np.NaN)
    else:
        r.append(i)

new.all_links = r
new = new.dropna()

In [10]:
new = new.reset_index()
#new_dic= dict(new)

#review_links = []
#for i in range(len(new.all_links)):
#    review_links.append(dict([
#('name', new["name"][i]),
#('links', new["all_links"][i])]))
    

#review_links

review_links = []
for i in range(len(new.all_links)):
    for x in range(len(new.all_links[i])):
        
        review_links.append(dict([
('name', new["name"][i]),
('links', new["all_links"][i][x])]))
    

review_links

[{'name': '1. Kebabsville',
  'links': 'https://www.tripadvisor.com//Restaurant_Review-g494941-d1371205-Reviews-Kebabsville-Indore_Indore_District_Madhya_Pradesh.html'},
 {'name': '1. Kebabsville',
  'links': 'https://www.tripadvisor.com//Restaurant_Review-g494941-d1371205-Reviews-or10-Kebabsville-Indore_Indore_District_Madhya_Pradesh.html'},
 {'name': '1. Kebabsville',
  'links': 'https://www.tripadvisor.com//Restaurant_Review-g494941-d1371205-Reviews-or20-Kebabsville-Indore_Indore_District_Madhya_Pradesh.html'},
 {'name': '1. Kebabsville',
  'links': 'https://www.tripadvisor.com//Restaurant_Review-g494941-d1371205-Reviews-or30-Kebabsville-Indore_Indore_District_Madhya_Pradesh.html'},
 {'name': '1. Kebabsville',
  'links': 'https://www.tripadvisor.com//Restaurant_Review-g494941-d1371205-Reviews-or40-Kebabsville-Indore_Indore_District_Madhya_Pradesh.html'},
 {'name': '1. Kebabsville',
  'links': 'https://www.tripadvisor.com//Restaurant_Review-g494941-d1371205-Reviews-or50-Kebabsville-I

In [11]:
new_df2 = pd.DataFrame(review_links)

l = new_df2["links"].to_list()

import numpy as np
r = []
for i in l:
    if(i=="https://www.tripadvisor.com/"):
        r.append(np.NaN)
    else:
        r.append(i)

new_df2["links"] = r
new_df2 = new_df2.dropna()
new_df2 = new_df2.reset_index()

In [12]:
review_links3 = []
for i in range(len(new_df2.links)):
    review_links3.append(dict([
('name', new_df2["name"][i]),
('links', new_df2["links"][i])]))

In [13]:
import time
import random
sleepTimes = [0.75, 0.5, 0.1,1,0.82,1.2,0.97]

name = []
rating = []
comments = []
summary = []
info = []
lin = []
users = []
i_d = []
for r in range(len(review_links3)):    
 #   for links in review_links[r]["links"]:
    soup = getAndParse(review_links3[r]["links"])
    time.sleep(random.choice(sleepTimes))
    comments+=([text.findNext().text for text in soup.findAll("span",class_="noQuotes")])
    users+=[text.text for text in soup.findAll("div",class_="info_text pointer_cursor")]
    for i in range(len(soup.findAll("div",class_="ui_column is-9"))): 
        s = soup.findAll("div",class_="ui_column is-9")
        rating+=([re.findall(r"(?: bubble_)+(\d{,2})",str(r)) for r in s[i].findAll("span",class_=re.compile("ui_bubble_rating bubble_"))]) 
        for x in range(len(s[i].findAll("span",class_=re.compile("ui_bubble_rating bubble_")))):
            name.append([(review_links3[r]["name"])])
            info.append([text.text for text in soup.findAll("div",class_= "cfvAV")])
            summary.append([text.text for text in soup.findAll("div",class_="cfvAVhjg")])
            lin.append([text.text for text in soup.findAll("a",class_="dOGcA Ci Wc _S C dkdrG")])
            i_d.append(x)

In [14]:
last_df = pd.DataFrame()
last_df["name"] = name
last_df["id"] = i_d
last_df["rating"] = rating
last_df["comments"] = comments

last_df["users"] = users
last_df["summary"] = summary
last_df["info"] = info
last_df["loc"] = lin
            
last_df.rating = last_df.rating.apply(lambda x : x[0])
last_df["name"] = last_df["name"].apply(lambda x : x[0])
#last_df["loc"] = last_df["loc"].apply(lambda x : x[0])

l = last_df["summary"].to_list()


import numpy as np
r = []
for i in l:
    if(i==[]):
        r.append([np.NaN])
    else:
        r.append(i)

last_df["summary"] = r

m = last_df["info"].to_list()


import numpy as np
k = []
for i in m:
    if(i==[]):
        k.append(np.NaN)
    else:
        k.append(i)

last_df["info"] = k
            
last_df.summary = last_df.summary.apply(lambda x : x[0])

In [15]:

last_df = pd.DataFrame()
last_df["name"] = name
last_df["id"] = i_d
last_df["rating"] = rating
last_df["comments"] = comments
last_df["users"] = users
last_df["summary"] = summary
last_df["info"] = info
last_df["loc"] = lin
            
last_df.rating = last_df.rating.apply(lambda x : x[0])
last_df["name"] = last_df["name"].apply(lambda x : x[0])
#last_df["loc"] = last_df["loc"].apply(lambda x : x[0])

l = last_df["summary"].to_list()


import numpy as np
r = []
for i in l:
    if(i==[]):
        r.append([np.NaN])
    else:
        r.append(i)

last_df["summary"] = r

m = last_df["info"].to_list()


import numpy as np
k = []
for i in m:
    if(i==[]):
        k.append(np.NaN)
    else:
        k.append(i)

last_df["info"] = k
            
last_df.summary = last_df.summary.apply(lambda x : x[0])

In [16]:
last_df.tail()

,name,id,rating,comments,users,summary,info,loc
475,9. Restaurant Shreemaya,0,50,Shrimaya is a well known name in Indore for it...,imrankhan3387,NaN,NaN,"[12 R N T Marg Hotel Shreemaya, Indore 452001 ..."
476,9. Restaurant Shreemaya,0,50,"This place is must visit for foodies, taste is...",Vishal J,NaN,NaN,"[12 R N T Marg Hotel Shreemaya, Indore 452001 ..."
477,9. Restaurant Shreemaya,0,40,"Well yes,by any standards its serves the best ...",pratyushj21,NaN,NaN,"[12 R N T Marg Hotel Shreemaya, Indore 452001 ..."
478,9. Restaurant Shreemaya,0,40,It was pretty crowded though as there were lot...,RAJESH S,NaN,NaN,"[12 R N T Marg Hotel Shreemaya, Indore 452001 ..."
479,9. Restaurant Shreemaya,0,50,1) Quality food\n2) Best hotel rooms\n3) Best ...,Dipanjan B,NaN,NaN,"[12 R N T Marg Hotel Shreemaya, Indore 452001 ..."


In [17]:
last_df.name.nunique()

10

In [18]:
# This connects us to the "legistlation" database, and the "collection" [think table] news 
# in that database
client = MongoClient()
client = MongoClient('mongodb://localhost:27017/')
db = client.restaurantDemoDB
collection = db.restaurant_reviews


In [19]:

# last_df=last_df.drop(columns=["info"])
last_df2 = last_df.reset_index()
data_dict = last_df2.to_dict("records")
collection.insert_many(data_dict)